In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [13]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [14]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 5ZHs7ZGXV45Y4oVyaddHPwRbvh4sgcRmDnx4Km6uud6A, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [6]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["id", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+------+----------------+---------+
|    | id   | airport_name   | LABEL   |
|----+------+----------------+---------|
|  0 | MKE  | Milwaukee      | Airport |
|  1 | ORD  | Chicago O-Hare | Airport |
|  2 | SJC  | San Jose       | Airport |
|  3 | DEN  | Denver         | Airport |
+----+------+----------------+---------+


In [7]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


+----+------------+----------+------------+------------+----------+
|    | START_ID   | END_ID   |   DISTANCE |   NUM_HOPS | TYPE     |
|----+------------+----------+------------+------------+----------|
|  0 | MKE        | ORD      |         66 |          1 | FLIES_TO |
|  1 | ORD        | MKE      |         66 |          1 | FLIES_TO |
|  2 | ORD        | DEN      |        886 |          1 | FLIES_TO |
|  3 | DEN        | ORD      |        886 |          1 | FLIES_TO |
|  4 | SJC        | DEN      |        948 |          1 | FLIES_TO |
|  5 | DEN        | SJC      |        948 |          1 | FLIES_TO |
|  6 | SJC        | ORD      |       1829 |          1 | FLIES_TO |
+----+------------+----------+------------+------------+----------+


In [15]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "id",
      id_space              = "Airport",  
      label                 = "Airport",  
      )

   df_importer.node_id_property_name("id")                     #  We need this method if/when we only insert nodes, [ then ] edges as a separate step
    
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()
    
print("--")


          0/? [?op/s]

          0/? [?op/s]

--


#  Diagnostics

In [16]:

#  What's in the graph ..

def f_print_graph(i_arg1):
    
   l_result = i_arg1.query("""
      MATCH ( n ) 
      RETURN n
      """)
   print(tabulate(l_result, headers='keys', tablefmt='psql'))
   
   l_result = i_arg1.query("""
      MATCH ( n ) - [ r ] -> ( m )
      RETURN r
      """)
   print(tabulate(l_result, headers='keys', tablefmt='psql'))


f_print_graph(my_graph)


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+--------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   | n.type   |
|----+-----------------+-------------+-----------+------------------+--------+----------|
|  0 |               0 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    | node     |
|  1 | 281474976710656 | ['Airport'] | Airport   | Milwaukee        | MKE    | node     |
|  2 | 281474976710657 | ['Airport'] | Airport   | San Jose         | SJC    | node     |
|  3 | 562949953421312 | ['Airport'] | Airport   | Denver           | DEN    | node     |
+----+-----------------+-------------+-----------+------------------+--------+----------+


          0/? [?op/s]

+----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   |
|----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------|
|  0 | 562949953421312 | ['Airport']    |               0 | FLIES_TO  |          886 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
|  1 | 281474976710656 | ['Airport']    |               1 | FLIES_TO  |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
|  2 |               0 | ['Airport']    | 281474976710656 | FLIES_TO  |           66 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
|  3 |               0 | ['Airport

#  at_version(), (snapshots) ..

In [17]:

#  graph.version
#
#     .  If we import both nodes and edges above, graph.version is usually 2.
#        If we import just nodes, then just edges, version is usually 4.
#        Basically the graph (versions) at each mutation. 
#
#        (version) is a number that constantly moves forward.
#
#     .  Print version, then mutate, and print version again.
#

l_vers_afterload    = my_graph.version
   #
display("Graph version after load: %d" % (l_vers_afterload))

l_query = """
   CREATE ( n: Restaurant { id: 'PAP' } )                     //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result = my_graph.query(l_query))


l_vers_afternode    = my_graph.version
   #
display("Graph version after node: %d" % (l_vers_afternode))


l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.id               = 'SJC'
   AND   m.id               = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result = my_graph.query(l_query))


l_vers_afteredge    = my_graph.version
   #
display("Graph version after edge: %d" % (l_vers_afteredge))


'Graph version after load: 2'

          0/? [?op/s]

'Graph version after node: 3'

          0/? [?op/s]

'Graph version after edge: 4'

In [19]:

#  Print the current graph, then print graph at a prior version
#
#     Used to be able to do,  my_graph2 = my_graph.at_version(n)
#     Now at_version() is not found.
#

#  This works perfectly;
#
#     .  We had a graph (Vers 2), then added 1 node (Vers 3), and separately
#        added 1 edge (Vers 4)
#
#     .  Vers 3 and Vers 4 of the graph correctly report only what it should
#

f_print_graph(my_graph)

print("")
print("#################################################################")
print("")

my_graph3 = my_graph.create_snapshot_at_version(l_vers_afternode)
f_print_graph(my_graph3)

print("")
print("#################################################################")
print("")

my_graph4 = my_graph.create_snapshot_at_version(l_vers_afteredge)
f_print_graph(my_graph4)


          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+--------+----------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_name   | n.id   | n.type   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+--------+----------+-----------------------------|
|  0 |               0 | ['Airport']    | Airport   | Chicago O-Hare   | ORD    | node     | nan                         |
|  1 | 281474976710658 | ['Restaurant'] | nan       | nan              | PAP    | node     | Pappadeauxs Seafood Kitchen |
|  2 | 281474976710657 | ['Airport']    | Airport   | San Jose         | SJC    | node     | nan                         |
|  3 | 281474976710656 | ['Airport']    | Airport   | Milwaukee        | MKE    | node     | nan                         |
|  4 | 562949953421312 | ['Airport']    | Airport   | Denver           | DEN    | node     | nan                         |
+----+----------

          0/? [?op/s]

+----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label    |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   | r.DRIVABLE   |
|----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------|
|  0 | 562949953421312 | ['Airport']    |               0 | FLIES_TO   |          886 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
|  1 | 281474976710656 | ['Airport']    |               1 | FLIES_TO   |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
|  2 |               0 | ['Airport']    | 281474976710657 | FLIES_TO   |         1829 |            1 | FLIES_TO | 2

          0/? [?op/s]

...

          0/? [?op/s]

          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+--------+----------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_name   | n.id   | n.type   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+--------+----------+-----------------------------|
|  0 |               0 | ['Airport']    | Airport   | Chicago O-Hare   | ORD    | node     | nan                         |
|  1 | 281474976710656 | ['Airport']    | Airport   | Milwaukee        | MKE    | node     | nan                         |
|  2 | 281474976710657 | ['Airport']    | Airport   | San Jose         | SJC    | node     | nan                         |
|  3 | 281474976710658 | ['Restaurant'] | nan       | nan              | PAP    | node     | Pappadeauxs Seafood Kitchen |
|  4 | 562949953421312 | ['Airport']    | Airport   | Denver           | DEN    | node     | nan                         |
+----+----------

          0/? [?op/s]

+----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label    |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   | r.DRIVABLE   |
|----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------|
|  0 | 562949953421312 | ['Airport']    |               0 | FLIES_TO   |          886 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
|  1 | 281474976710656 | ['Airport']    |               1 | FLIES_TO   |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
|  2 |               0 | ['Airport']    | 281474976710657 | FLIES_TO   |         1829 |            1 | FLIES_TO | 2

In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())

print("")

if 'my_graph3' in globals():                    #  Currently ignores version; reports on latest graph version
   display(my_graph3.num_nodes())
   display(my_graph2.num_edges())
else: 
   print("--")


In [ ]:

#  Print available snapshots for a given graph
#

print(my_graph.get_snapshots())

#  Error,
#
#     AttributeError: '_Database' object has no attribute 'graphs'


In [ ]:

display(my_graph.snapshot_of)
display(my_graph2.snapshot_of)